In [1]:
!pip install pennylane

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 53.7 MB/s eta 0:00:00


In [16]:
import pennylane as qml
import torch
import numpy as np
import random
import torch.nn as nn
from tqdm import tqdm

In [45]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [46]:
path='/content/gdrive/MyDrive/New_test/'

In [27]:
import tensorflow as tf 
from tensorflow.keras.preprocessing import sequence

vocab_size = 1000
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=vocab_size)

class_names = ["Negative", "Positive"]

17464789/17464789 [==============================] - 0s 0us/step


In [29]:
# Select dataset evenly 
selected_num=128

new_x_train=[]
new_y_train=[]
indx=0
while len(new_x_train) < selected_num:
    if len(x_train[indx]) <= 200:
        sum_y=np.sum(new_y_train)
        len_y=len(new_y_train)
        if sum_y >= selected_num//2:
            if y_train[indx]==0:
                new_x_train.append(x_train[indx])
                new_y_train.append(y_train[indx])
                indx+=1
            else:
                indx+=1
                continue
        elif ((len_y-sum_y) >= selected_num//2):
            if y_train[indx]==1:
                new_x_train.append(x_train[indx])
                new_y_train.append(y_train[indx])
                indx+=1
            else:
                indx+=1
                continue
        else:
            new_x_train.append(x_train[indx])
            new_y_train.append(y_train[indx])
            indx+=1
    else:
        indx+=1

selected_num=128

new_x_test=[]
new_y_test=[]
indx=0
while len(new_x_test) < selected_num:
    if len(x_test[indx]) <= 200:
        sum_y=np.sum(new_y_test)
        len_y=len(new_y_test)
        if sum_y >= selected_num//2:
            if y_test[indx]==0:
                new_x_test.append(x_test[indx])
                new_y_test.append(y_test[indx])
                indx+=1
            else:
                indx+=1
                continue
        elif ((len_y-sum_y) >= selected_num//2):
            if y_test[indx]==1:
                new_x_test.append(x_test[indx])
                new_y_test.append(y_test[indx])
                indx+=1
            else:
                indx+=1
                continue
        else:
            new_x_test.append(x_test[indx])
            new_y_test.append(y_test[indx])
            indx+=1
    else:
        indx+=1

In [30]:
x_train=np.array(new_x_train)#[:128]
y_train=np.array(new_y_train)#[:128]

x_test=np.array(new_x_test)#[:128]
y_test=np.array(new_y_test)#[:128]

<ipython-input-30-b98d59874a29>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_train=np.array(new_x_train)#[:128]
<ipython-input-30-b98d59874a29>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_test=np.array(new_x_test)#[:128]


In [31]:
# Get the word index from the dataset
word_index = tf.keras.datasets.imdb.get_word_index()

# Ensure that "special" words are mapped into human readable terms 
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNKNOWN>"] = 2
word_index["<UNUSED>"] = 3

# Perform reverse word lookup and make it callable
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

1641221/1641221 [==============================] - 0s 0us/step


In [32]:
# Concatonate test and training datasets
allreviews = np.concatenate((x_train, x_test), axis=0)

# Review lengths across test and training whole datasets
print("Maximum review length: {}".format(len(max((allreviews), key=len))))
print("Minimum review length: {}".format(len(min((allreviews), key=len))))
result = [len(x) for x in allreviews]
print("Mean review length: {}".format(np.mean(result)))

# Print a review and it's class as stored in the dataset. Replace the number
# to select a different review.
print("")
print("Machine readable Review")
print("  Review Text: " + str(x_train[60]))
print("  Review Sentiment: " + str(y_train[60]))

# Print a review and it's class in human readable format. Replace the number
# to select a different review.
print("")
print("Human Readable Review")
print("  Review Text: " + decode_review(x_train[0]))
print("  Review Sentiment: " + class_names[y_train[0]])

Maximum review length: 200
Minimum review length: 19
Mean review length: 132.546875

Machine readable Review
  Review Text: [1, 13, 244, 6, 666, 337, 7, 2, 2, 5, 28, 296, 958, 7, 27, 102, 366, 150, 5, 600, 7, 98, 28, 2, 8, 97, 72, 462, 2, 122, 89, 8, 2, 369, 5, 2, 84, 10, 10, 14, 20, 9, 2, 44, 6, 132, 52, 17, 2, 84, 125, 190, 29, 47, 35, 2, 270, 7, 2, 15, 2, 90, 39, 399, 183, 15, 238, 43, 97, 90, 804, 21, 11, 4, 130, 29, 630, 56, 399, 98, 4, 2, 7, 113, 10, 10, 4, 20, 9, 73, 2, 19, 486, 883, 52, 116, 5, 82, 6, 227, 7, 6, 2, 837, 94, 31, 7, 148, 102, 121, 25, 100, 43, 30, 654, 54, 12, 630]
  Review Sentiment: 1

Human Readable Review
  Review Text: <START> big <UNKNOWN> big <UNKNOWN> bad music and a <UNKNOWN> <UNKNOWN> <UNKNOWN> these are the words to best <UNKNOWN> this terrible movie i love cheesy horror movies and i've seen <UNKNOWN> but this had got to be on of the worst ever made the plot is <UNKNOWN> <UNKNOWN> and ridiculous the acting is an <UNKNOWN> the script is completely <UNKN

In [33]:
# The length of reviews
review_length = 200

# Padding / truncated our reviews
x_train = sequence.pad_sequences(x_train, maxlen = review_length)
x_test = sequence.pad_sequences(x_test, maxlen = review_length)

# Check the size of our datasets. Review data for both test and training should 
# contain 25000 reviews of 500 integers. Class data should contain 25000 values, 
# one for each review. Class values are 0 or 1, indicating a negative 
# or positive review.
print("Shape Training Review Data: " + str(x_train.shape))
print("Shape Training Class Data: " + str(np.shape(y_train)))
print("Shape Test Review Data: " + str(x_test.shape))
print("Shape Test Class Data: " + str(y_test.shape))

# Note padding is added to start of review, not the end
print("")
print("Human Readable Review Text (post padding): " + decode_review(x_train[0]))

Shape Training Review Data: (128, 200)
Shape Training Class Data: (128,)
Shape Test Review Data: (128, 200)
Shape Test Class Data: (128,)

Human Readable Review Text (post padding): <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <START> big <UNKNOWN> big <UNKNOWN> bad music and a <UNKNOWN> <UNKNOWN> <UNKNOWN> these are the words to best <UNKNOWN> this terrible movie i love cheesy horror movies and i've seen <UNKNOWN> but this had got to be on of the worst ever made the plot is <UNKNOWN> <UNKNOWN> and ridiculous the acting is an <UNKNOWN> the script is completely <UNKNOWN> the best is the end <UNKNOWN> with the <UNKNOWN> and how he worked out who the killer is it's just so <UNKNOWN> <UNKNOWN> written the <UNKNOWN> are <UNKNOWN> and funny in <UNKNOWN> <UNKNOWN> the <UNKNOWN> is big lots of <UNKNOWN> <UNKNOWN> men <UNKNOWN> those cut <UNKNOWN> <UNKNOWN> that show off their <UNKNOWN> <UNKNOWN> that men actually <UNKNOWN> them and the music is just <UNKNOWN> <UNKNOWN> tha

In [34]:
new_X_train=[]
for seq in x_train:
    a=torch.tensor(np.pi*(seq/1000-0.5))
    new_X_train.append(torch.reshape(a, (10, 20)))

new_y_train=[]
for seq in y_train:
    new_y_train.append(torch.tensor(seq))

In [35]:
new_X_train=torch.stack(new_X_train)

label=torch.stack(new_y_train)

In [36]:
new_X_test=[]
for seq in x_test:
    a=torch.tensor(np.pi*(seq/1000-0.5))
    new_X_test.append(torch.reshape(a, (10, 20)))

new_y_test=[]
for seq in y_test:
    new_y_test.append(torch.tensor(seq))

In [3]:
import functools
import inspect
import math
from collections.abc import Iterable
from typing import Callable, Dict, Union, Any

from pennylane.qnode import QNode

try:
    import torch
    from torch.nn import Module

    TORCH_IMPORTED = True
except ImportError:
    # The following allows this module to be imported even if PyTorch is not installed. Users
    # will instead see an ImportError when instantiating the TorchLayer.
    from unittest.mock import Mock

    Module = Mock
    TORCH_IMPORTED = False


class TorchLayer(Module):
    def __init__(self,qnode,weights):
        if not TORCH_IMPORTED:
            raise ImportError(
                "TorchLayer requires PyTorch. PyTorch can be installed using:\n"
                "pip install torch\nAlternatively, "
                "visit https://pytorch.org/get-started/locally/ for detailed "
                "instructions."
            )
        super().__init__()

        #weight_shapes = {
        #    weight: (tuple(size) if isinstance(size, Iterable) else () if size == 1 else (size,))
        #    for weight, size in weight_shapes.items()
        #}

        # validate the QNode signature, and convert to a Torch QNode.
        # TODO: update the docstring regarding changes to restrictions when tape mode is default.
        #self._signature_validation(qnode, weight_shapes)
        self.qnode = qnode
        self.qnode.interface = "torch"

        self.qnode_weights = weights

    def forward(self, inputs):  # pylint: disable=arguments-differ
        """Evaluates a forward pass through the QNode based upon input data and the initialized
        weights.

        Args:
            inputs (tensor): data to be processed

        Returns:
            tensor: output data
        """

        if len(inputs.shape) > 1:
            # If the input size is not 1-dimensional, unstack the input along its first dimension,
            # recursively call the forward pass on each of the yielded tensors, and then stack the
            # outputs back into the correct shape
            reconstructor = [self.forward(x) for x in torch.unbind(inputs)]
            return torch.stack(reconstructor)

        # If the input is 1-dimensional, calculate the forward pass as usual
        return self._evaluate_qnode(inputs)


    def _evaluate_qnode(self, x):
        """Evaluates the QNode for a single input datapoint.

        Args:
            x (tensor): the datapoint

        Returns:
            tensor: output datapoint
        """
        kwargs = {
            **{self.input_arg: x},
            **{arg: weight.to(x) for arg, weight in self.qnode_weights.items()},
        }
        res = self.qnode(**kwargs)

        if isinstance(res, torch.Tensor):
            return res.type(x.dtype)

        return torch.hstack(res).type(x.dtype)

    def __str__(self):
        detail = "<Quantum Torch Layer: func={}>"
        return detail.format(self.qnode.func.__name__)

    __repr__ = __str__
    _input_arg = "inputs"

    @property
    def input_arg(self):
        """Name of the argument to be used as the input to the Torch layer. Set to ``"inputs"``."""
        return self._input_arg

In [4]:
class QSAL_pennylane(torch.nn.Module):
    def __init__(self,S,n,Denc,D):
        super().__init__()
        self.seq_num=S
        self.init_params_Q=torch.nn.Parameter(torch.stack([(np.pi/4) * (2 * torch.randn(n*(D+2)) - 1) for _ in range(self.seq_num)]))
        self.init_params_K=torch.nn.Parameter(torch.stack([(np.pi/4) * (2 * torch.randn(n*(D+2)) - 1) for _ in range(self.seq_num)]))
        self.init_params_V=torch.nn.Parameter(torch.stack([(np.pi/4) * (2 * torch.randn(n*(D+2)) - 1) for _ in range(self.seq_num)]))
        self.num_q=n
        self.Denc=Denc
        self.D=D
        self.d=n*(Denc+2)
        self.dev = qml.device("default.qubit", wires=self.num_q)
        
        self.vqnod=qml.QNode(self.circuit_v, self.dev, interface="torch")
        self.qnod=qml.QNode(self.circuit_qk, self.dev, interface="torch")
        self.weight_v = [{"weights": self.init_params_V[i]} for i in range(self.seq_num)]
        self.weight_q = [{"weights": self.init_params_Q[i]} for i in range(self.seq_num)]
        self.weight_k = [{"weights": self.init_params_K[i]} for i in range(self.seq_num)]
        #self.v_linear ={} #[qml.qnn.TorchLayer(self.vqnod[i], self.weight_shapes) for i in range(self.seq_num)]
        #for i in range(self.seq_num):
        self.v_linear = [TorchLayer(self.vqnod, self.weight_v[i]) for i in range(self.seq_num)]
        self.q_linear = [TorchLayer(self.qnod, self.weight_q[i]) for i in range(self.seq_num)]
        self.k_linear = [TorchLayer(self.qnod, self.weight_k[i]) for i in range(self.seq_num)]
        #self.qqnod=[qml.QNode(self.circuit_qk, self.dev, interface="torch") for i in range(self.seq_num)]

    def random_op(self):
        a=random.randint(0, 4)
        if a==0:
            op=qml.Identity(0)
        elif a==1:
            op=qml.PauliX(0)
        elif a==2:
            op=qml.PauliY(0)
        else:
            op=qml.PauliZ(0)

        op_elimated=qml.Identity(0)
        for i in range(1,self.num_q):
            op_elimated=op_elimated@qml.Identity(i)
        Select_wrong=True
        while Select_wrong:
            for i in range(1,self.num_q):
                a=random.randint(0, 4)
                if a==0:
                    op=op@qml.Identity(i)
                elif a==1:
                    op=op@qml.PauliX(i)
                elif a==2:
                    op=op@qml.PauliY(i)
                else:
                    op=op@qml.PauliZ(i)
            if op!=op_elimated:
                Select_wrong=False
        return op

    def circuit_v(self,inputs,weights):
            op=self.random_op()
            # feature_map
            indx=0
            for j in range(self.num_q):
                qml.RX(inputs[indx],j)
                qml.RY(inputs[indx+1],j)
                indx+=2
            for i in range(self.Denc):
                for j in range(self.num_q):
                    qml.CNOT(wires=(j,(j+1)%self.num_q))

                for j in range(self.num_q):
                    qml.RY(inputs[indx],j)
                    indx+=1
            # Ansatz
            indx=0
            for j in range(self.num_q):
                qml.RX(weights[indx],j)
                qml.RY(weights[indx+1],j)
                indx+=2
            for i in range(self.D):
                for j in range(self.num_q):
                    qml.CNOT(wires=(j,(j+1)%self.num_q))
                    
                for j in range(self.num_q):
                    #qc.rx(params[indx],j)
                    qml.RY(weights[indx],j)
                    indx+=1
            return [qml.expval(op) for i in range(self.d)] 

    def circuit_qk(self,inputs,weights):
        op=self.random_op()
        # feature_map
        indx=0
        for j in range(self.num_q):
            qml.RX(inputs[indx],j)
            qml.RY(inputs[indx+1],j)
            indx+=2
        for i in range(self.Denc):
            for j in range(self.num_q):
                qml.CNOT(wires=(j,(j+1)%self.num_q))

            for j in range(self.num_q):
                qml.RY(inputs[indx],j)
                indx+=1
        # Ansatz
        indx=0
        for j in range(self.num_q):
            qml.RX(weights[indx],j)
            qml.RY(weights[indx+1],j)
            indx+=2
        for i in range(self.D):
            for j in range(self.num_q):
                qml.CNOT(wires=(j,(j+1)%self.num_q))
                
            for j in range(self.num_q):
                #qc.rx(params[indx],j)
                qml.RY(weights[indx],j)
                indx+=1
        return [qml.expval(qml.PauliZ(0))]

    def forward(self,input):

        Q_output=torch.stack([self.q_linear[i](input[:,i]) for i in range(self.seq_num)])
        K_output=torch.stack([self.k_linear[i](input[:,i]) for i in range(self.seq_num)])
        V_output=torch.stack([self.v_linear[i](input[:,i]) for i in range(self.seq_num)])
        
        batch_size=len(input)
        Q_output=Q_output.transpose(0,2).repeat((self.seq_num,1,1))
        K_output=K_output.transpose(0,2).repeat((self.seq_num,1,1)).transpose(0,2)
        #print(V_output.size())
        #Q_grid, K_grid=torch.meshgrid(Q_output, K_output, indexing='ij')
        alpha=torch.exp(-(Q_output-K_output)**2)
        alpha=alpha.transpose(0,1)
        V_output=V_output.transpose(0,1)
        output=[]

        for i in range(self.seq_num):
            
            Sum_a=torch.sum(alpha[:,i,:],-1)
            div_sum_a=(1/Sum_a).repeat(self.d,self.seq_num,1).transpose(0,2)
            
            Sum_w=torch.sum(alpha[:,:,i].repeat((self.d,1,1)).transpose(0,2).transpose(0,1)*V_output*div_sum_a,1)
            output.append(Sum_w)
        return input+torch.stack(output).transpose(0,1)

class QSANN_pennylane(torch.nn.Module):
    def __init__(self,S,n,Denc,D,num_layers):
        """
        # input: input data
        # weight: trainable parameter
        # n: # of of qubits
        # d: embedding dimension which is equal to n(Denc+2)
        # Denc: the # number of layers for encoding 
        # D: the # of layers of variational layers
        # type "K": key, "Q": Query, "V": value
        """
        super().__init__()
        self.qsal_lst=[QSAL_pennylane(S,n,Denc,D) for _ in range(num_layers)]
        self.qnn=nn.Sequential(*self.qsal_lst)

    def forward(self,input):

        return self.qnn(input)

class QSANN_text_classifier(torch.nn.Module):
    def __init__(self,S,n,Denc,D,num_layers):
        """
        # input: input data
        # weight: trainable parameter
        # n: # of of qubits
        # d: embedding dimension which is equal to n(Denc+2)
        # Denc: the # number of layers for encoding 
        # D: the # of layers of variational layers
        # type "K": key, "Q": Query, "V": value
        """
        super().__init__()
        self.Qnn=QSANN_pennylane(S,n,Denc,D,num_layers)
        self.final_layer=nn.Linear(n*(Denc+2)*S, 1)
        self.final_layer=self.final_layer.float()

    def forward(self,input):

        x=self.Qnn(input)
        x=torch.flatten(x,start_dim=1)
        
        return torch.sigmoid(self.final_layer(x))

# Toy data

In [20]:
X=[[np.array([random.random() for _ in range(6)]) for _ in range(5)] for _ in range(32)]
Y=[random.randint(0,1) for _ in range(32)]

In [21]:
model=QSANN_text_classifier(5,2,1,1,2)

In [22]:
optimizer = torch.optim.Adam(lr=0.03, params=model.parameters())

In [23]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(trainable_params)

211


In [24]:
criterion = torch.nn.CrossEntropyLoss()

In [25]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = (torch.round(torch.sign(preds-0.5))+1)//2
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [26]:
for iepoch in tqdm(range(10)):
    optimizer.zero_grad()
    X_tensor=torch.tensor(X)
    predictions=model(X_tensor.float()).squeeze(1)
    #predictions=torch.sign(predictions)
    label=torch.tensor(Y)
    loss = criterion(predictions, label.float())
    acc = binary_accuracy(predictions, label)
    print('')
    print('Accuracy:',acc)
    print('')
    print(loss)
    loss.backward()
    optimizer.step()

  0%|          | 0/10 [00:00<?, ?it/s]


Accuracy: tensor(0.5312)

tensor(51.8366, grad_fn=<DivBackward1>)


 10%|█         | 1/10 [00:21<03:11, 21.28s/it]


Accuracy: tensor(0.5625)

tensor(51.4881, grad_fn=<DivBackward1>)


 20%|██        | 2/10 [00:43<02:54, 21.85s/it]

...

 80%|████████  | 8/10 [02:58<00:45, 22.61s/it]


Accuracy: tensor(0.7188)

tensor(50.7651, grad_fn=<DivBackward1>)


 90%|█████████ | 9/10 [03:21<00:22, 22.72s/it]


Accuracy: tensor(0.6250)

tensor(51.0952, grad_fn=<DivBackward1>)


100%|██████████| 10/10 [03:44<00:00, 22.45s/it]


# Real Data

In [37]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = (torch.round(torch.sign(preds-0.5))+1)//2
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [38]:
model = QSANN_text_classifier(10,2,8,1,1)

In [39]:
optimizer = torch.optim.Adam(lr=0.005, params=model.parameters())
criterion = torch.nn.CrossEntropyLoss()

In [40]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(trainable_params)

381


In [41]:
for iepoch in tqdm(range(40)):
    
    optimizer.zero_grad()

    idx = torch.randperm(128)[:32]
    X_batch = new_X_train[idx]
    predictions=model(X_batch.float()).squeeze(1)

    label_batch=label[idx]
    loss = criterion(predictions, label_batch.float())
    acc = binary_accuracy(predictions,label_batch)
    print('')
    print('Accuracy:',acc)
    print('')
    print(loss)
    loss.backward()
    optimizer.step()

  0%|          | 0/40 [00:00<?, ?it/s]


Accuracy: tensor(0.5000)

tensor(59.0043, grad_fn=<DivBackward1>)


  2%|▎         | 1/40 [00:48<31:23, 48.30s/it]


Accuracy: tensor(0.5312)

tensor(58.5009, grad_fn=<DivBackward1>)


  5%|▌         | 2/40 [01:29<28:07, 44.41s/it]

...

 95%|█████████▌| 38/40 [27:03<01:25, 42.93s/it]


Accuracy: tensor(0.7812)

tensor(53.6991, grad_fn=<DivBackward1>)


 98%|█████████▊| 39/40 [27:45<00:42, 42.80s/it]


Accuracy: tensor(0.9062)

tensor(46.1245, grad_fn=<DivBackward1>)


100%|██████████| 40/40 [28:27<00:00, 42.69s/it]


In [42]:
X_batch = new_X_train
predictions=model(X_batch.float()).squeeze(1)

loss = criterion(predictions, label.float())
acc = binary_accuracy(predictions,label)
print('')
print('Accuracy:',acc)
print('')
print(loss)


Accuracy: tensor(0.8438)

tensor(302.8917, grad_fn=<DivBackward1>)


In [43]:
label_test=torch.stack(new_y_test)

In [44]:
X_batch = torch.stack(new_X_test)
predictions=model(X_batch.float()).squeeze(1)

loss = criterion(predictions, label_test.float())
acc = binary_accuracy(predictions,label_test)
print('')
print('Accuracy:',acc)
print('')
print(loss)


Accuracy: tensor(0.4062)

tensor(312.7864, grad_fn=<DivBackward1>)


In [47]:
file_name=path+'QSANN_pennylane_model'
torch.save(model.state_dict(),file_name)